In [ ]:
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install torch
!pip install datasets

# Generating CN

In [ ]:
from google.colab import drive
from IPython.display import clear_output
clear_output()

In [ ]:
import os
import json
import csv
import pandas
import time
import random
from collections import Counter
from statistics import mean, median, stdev
import numpy as np
import matplotlib.pyplot as plt
import sys

from generate import *

df = pandas.read_csv(path_to_hscn)
val_df = df[df.Assigned_to == 'test'].reset_index(drop=True)

# #156 is the longest HS-CN text, and it will be used as max_len for both encoder and decoder
max_len=156
batch_size=20

________
# with GPT-2

In [ ]:
model_type = 'GPT-2'
generated_cns = generate_cn(model_type, model_path, list_hs = df['HS'], decoding = 'top-p', p=0.92, k=40, nb=5, rp=2.0, specialtokens=True, max_len =156)
prepare_output_df(generated_cns, df, model_type, 'tp')

________
# with DialoGPT


In [ ]:
model_type = 'DialoGPT'
generated_cns = generate_cn(model_type, model_path, list_hs = df['HS'], decoding = 'top-p', p=0.92, k=40, nb=5, rp=2.0, specialtokens=True, max_len =156)
prepare_output_df(generated_cns, df, model_type, 'tp')

___
# with BERT


In [ ]:
model_type = 'BERT'
generated_cns = generate_cn(model_type, model_path, list_hs = df['HS'], decoding = 'top-p', p=0.92, k=40, nb=5, rp=2.0, specialtokens=True, max_len =156)
prepare_output_df(generated_cns, df, model_type, 'tp')

_____
# with BART

In [ ]:
model_type = 'BART'
generated_cns = generate_cn(model_type, model_path, list_hs = df['HS'], decoding = 'top-p', p=0.92, k=40, nb=5, rp=2.0, specialtokens=True, max_len =156)
prepare_output_df(generated_cns, df, model_type, 'tp')

___
# with T5

In [ ]:
model_type = 'T5'
generated_cns = generate_cn(model_type, model_path, list_hs = df['HS'], decoding = 'top-p', p=0.92, k=40, nb=5, rp=2.0, specialtokens=True, max_len =156)
prepare_output_df(generated_cns, df, model_type, 'tp')